## Задание 1

Реализовать генератор матрциц, который должен поддерживать функции:
* Генерация абсолютно случайной матрицы $n\times m$
* Генерация случайной диагональной матрицы $n\times n$
* Генерация случайной верхнетреугольной матрицы
* Генерация случайной нижнетреугольной матрицы
* Генерация симметричной матрицы
* Генерация вырожденной матрицы
* Генерация матрицы ступенчатого вида $n\times n$ ранга $m$
* Генерация возмущения матрицы $n\times m$, каждый элемент которой не превосходит по модулю заданный $\varepsilon$. Оценить величину нормы матрицы возмущений в зависимости от параметра $\varepsilon$ (оценить верхную границу).

Оценить численно вероятность того, что созданная матрица будет вырожденной для какого-либо случая выше. 


In [110]:
# Реализую произвольную m,n, верхнетреугольную и симметричную. Остальное на вас - вам нужно дописать функцию.
# Не забудьте откомментировать ваши изменения в документации к функции!

import numpy as np
from itertools import *

def matrix_generate(rows, columns, type_ = "full", eps = 0):
    """
    matrix_generate(rows, columns, type_ = "full")
    
    Создаёт случайную матрицу выбранного типа. 
    
    Если матрицу нужных размеров создать нельзя должен выдать
    строку f"Error with type {type_} and shape ({rows},{columns})".
    
    Parameters
    ----------
    
    rows : int
        Количество строк в создаваемой матрице.
    columns : int
        Количество столбцов в создаваемой матрице.
    type_ : str, optional
        Тип создаваемой матрицы: "full", "upper_triangular", "symmetric" и т.д.
    eps: float, optional
        Дополнительное число, использующееся при генерации для некоторых типов матриц.
    
    Returns
    -------
    out : ndarray or str
        Выдаёт матрицу нужного типа либо ошибку.
        
    Notes
    -----
    Поддерживаемые типы матриц:
        "full","upper_triangular",
        "symmetric",
        "diagonal", "lower_triangular",
        "degenerate", "stagging",
        "perturbation"
        ...
    
    
    """
    
    A = None
    
    if type_ == "full":
        
        A = np.random.random(size=(rows, columns))
        
    elif type_ == "upper_triangular":
        
        A = np.random.random(size=(rows, columns))

        for i in range(rows):
            for j in range(columns):
                if (i > j):
                    A[i, j] = 0
        
    elif type_ == "lower_triangular":
        
        A = matrix_generate(columns, rows, type_ = "upper_triangular", eps = 0)
        
        A = np.transpose(A)
        
    elif type_ == "diagonal":
        
        if rows != columns:
            return f"Error with type {type_} and shape ({rows},{columns})"
        
        else:
            
            A = np.random.random(size=(rows, columns))
        
            for i in range(rows):
                for j in range(columns):
                    if (i != j):
                        A[i, j] = 0
                        
            # Здесь вероятность вырожденности равна вероятности получить ноль хотя бы в одной ячейке диагонали. 
            # np.random.random выдает число типа float из полуинтервала [0.0, 1.0).
            # Это числа без знака и с нулевым порядком, то есть остаётся только мантисса, которая до 23 бит. 
            # Тогда искомая вероятность -- это $ n * 2^{-23}$.
            # Заметим, что вероятность для верхнетреугольной и нижнетреугольной определяется таким же образом.
        
    elif type_ == "symmetric":
        
        if rows != columns:
            return f"Error with type {type_} and shape ({rows},{columns})"
        
        A = matrix_generate(columns, rows, type_ = "upper_triangular", eps = 0)
        
        A += np.transpose(A) 
        
    elif type_ == "degenerate":
        
        if rows != columns:
            return f"Error with type {type_} and shape ({rows},{columns})"
        
        A = np.random.random(size=(rows, columns))
        b = np.random.random(size=rows)
        ind = np.random.randint(rows)
        
        A[ind] = A @ b - A[ind] * b[ind]
        # Здесь вероятность того, что матрица будет вырожденной, равна 1
        
    elif type_ == "stagging":
        
        if rows < columns:
            return f"Error with type {type_} and shape ({rows},{columns})"
        
        A = np.random.random(size=(rows, rows))
        b = sorted(np.random.choice(rows, columns, replace=False))
        b = np.pad(b, (0, rows-columns), mode='constant')
        
        for i in range(rows):
            for j in range(rows):
                if (i >= columns):
                    A[i, j] = 0
                elif (i < columns):
                    if (j < b[i]):
                        A[i, j] = 0
                        
    elif type_ == "perturbation":
        
        A = (2 * np.random.random(size=(rows, columns)) - 1) * eps
        # Оценим бесконечность нормой, тогда ||A|| < rows * eps
        # Аналогично для 1-нормы, ||A|| < columns * eps
    
    return A

In [111]:
matrix_generate(1, 3)

array([[0.87113969, 0.9869859 , 0.1311559 ]])

In [112]:
matrix_generate(4, 4, type_ = "upper_triangular")

array([[0.89336848, 0.52651441, 0.69232337, 0.58348449],
       [0.        , 0.45879803, 0.19554831, 0.98176107],
       [0.        , 0.        , 0.32042206, 0.25098292],
       [0.        , 0.        , 0.        , 0.45133909]])

In [113]:
matrix_generate(4, 3, type_ = "upper_triangular")

array([[0.19660557, 0.7938302 , 0.10270976],
       [0.        , 0.19762084, 0.65439083],
       [0.        , 0.        , 0.47293649],
       [0.        , 0.        , 0.        ]])

In [114]:
matrix_generate(4, 4, type_ = "symmetric")

array([[1.06680708, 0.90381023, 0.54031987, 0.3876856 ],
       [0.90381023, 0.12639696, 0.45256298, 0.78391648],
       [0.54031987, 0.45256298, 1.26093444, 0.34736421],
       [0.3876856 , 0.78391648, 0.34736421, 0.45870675]])

In [115]:
matrix_generate(4, 1, type_ = "symmetric")

'Error with type symmetric and shape (4,1)'

In [116]:
matrix_generate(4, 3, type_ = "lower_triangular")

array([[0.80630944, 0.        , 0.        ],
       [0.72545594, 0.66335674, 0.        ],
       [0.48358867, 0.86607375, 0.11504583],
       [0.87330996, 0.73341417, 0.93840884]])

In [118]:
matrix_generate(4, 4, type_ = "diagonal")

array([[0.98914136, 0.        , 0.        , 0.        ],
       [0.        , 0.00443455, 0.        , 0.        ],
       [0.        , 0.        , 0.11658575, 0.        ],
       [0.        , 0.        , 0.        , 0.96061163]])

In [119]:
A = matrix_generate(2, 2, type_ = "degenerate")
print(A)

[[0.84130333 0.89258569]
 [0.88508639 0.94869753]]


In [120]:
np.linalg.det(A)

0.00812694114812551

In [122]:
matrix_generate(5, 4, type_ = "stagging")

array([[0.43098109, 0.82445247, 0.81399525, 0.21169101, 0.29542204],
       [0.        , 0.73513557, 0.32882643, 0.27550424, 0.38445565],
       [0.        , 0.        , 0.80721327, 0.49394477, 0.463681  ],
       [0.        , 0.        , 0.        , 0.        , 0.74923553],
       [0.        , 0.        , 0.        , 0.        , 0.        ]])

In [124]:
matrix_generate(4, 5, type_ = "perturbation", eps = 0.1)

array([[-0.03170393, -0.04017835, -0.05951135,  0.07599368, -0.09504901],
       [ 0.04838783, -0.04806761, -0.07859954, -0.05837268, -0.05849822],
       [-0.00257696,  0.09951771, -0.06476353,  0.08815343, -0.03237491],
       [ 0.02777243,  0.02747127, -0.06517267, -0.0081918 ,  0.07607488]])

### Задание 2

Реализовать вычисление трех основных норм векторов (L1, L2 и максимальную) и подчиненных им матричных норм. Реализовать вычисление числа обусловленности.

Примечание: для вычисления собственных значений можно использовать linalg.eigvals из модуля scipy.

In [125]:
import numpy as np
from scipy import linalg

def vec_L1_norm(x):
    return sum(abs(x))

def vec_L2_norm(x):
    return np.sqrt(sum(x**2))

def vec_max_norm(x):
    return max(abs(x))

def mat_L1_norm(X):
    return max(abs(X) @ np.ones(np.shape(X)[1]))

def mat_max_norm(X):
    return max(np.ones(np.shape(X)[0]) @ abs(X))
    # А можно было return mat_L1_norm(np.transpose(X))
    
def mat_L2_norm(X):
    return np.sqrt(max(linalg.eigvals(np.transpose(X) @ X)))

def conditionality_L1(X):
    return mat_L1_norm(np.linalg.inv(X)) * mat_L1_norm(X)

def conditionality_max(X):
    return conditionality_L1(np.transpose(X))

def conditionality_L2(X):
    eigs = linalg.eigvals(np.transpose(X) @ X)
    return np.sqrt(max(eigs) / min(eigs))

In [126]:
x = np.random.random(5)
X = matrix_generate(3, 3)
print(x)
print(X)
print(vec_L1_norm(x))
print(vec_L2_norm(x))
print(vec_max_norm(x))
print(mat_L1_norm(X))
print(mat_L2_norm(X))
print(mat_max_norm(X))
print(conditionality_L1(X))
print(conditionality_max(X))
print(conditionality_L2(X))

[0.90176636 0.20673895 0.32879104 0.76753681 0.34515829]
[[0.04836387 0.88043327 0.78518878]
 [0.05888168 0.95263364 0.07528802]
 [0.85217985 0.49861975 0.37700187]]
2.5499914557210626
1.2931643787938707
0.9017663551379953
1.7278014687208134
(1.6344169861329694+0j)
2.33168665717188
4.6568405058081686
5.961810534495832
(3.4797069881861917+0j)


### Задание 3

Реализовать метод Гаусса приведения матрицы к ступенчатому виду. Реализовать функцию вычисления ранга матрицы. Сгенерировать вырожденные матрицы различных рангов и размеров и проверить алгоритм.

In [127]:
def ForwardGauss(X, rank = False):
    n, m = X.shape
    string = 0
    shift = 0
    for i in range(m):
        # print(np.abs(X.T[column + shift][column:]))
        if (string < n):
            ind = string + np.argmax(np.abs(X.T[string + shift][string:]))
        else: continue
    
        if X.T[string + shift][ind] != 0:
            X[string], X[ind] = X[ind], X[string]
        else:
            shift += 1
            continue

        for j in range(string + 1, n):
            X[j] -=  X[string] * X[j, i] / X[string, i]
        string += 1
        
    if rank:
        rk = n - shift
        return X, rk
    else:
        return X
    
def rank(X):
    return ForwardGauss(X, rank = True)[1]

In [128]:
X = np.random.randint(6, size=(4, 4))
print(X)

[[4 0 3 4]
 [1 4 3 5]
 [2 3 4 3]
 [4 1 2 5]]


In [129]:
print(X[0])
print(X.T[0])
print(X[2, 0])
print(X[0, 2])

[4 0 3 4]
[4 1 2 4]
2
3


In [130]:
print(X.T[2][1:])

[3 4 2]


In [131]:
X = matrix_generate(4, 4)
print(X)
X = ForwardGauss(X)
print(X)
print(rank(X))

[[4.79775059e-01 4.80457748e-04 2.55798208e-01 6.31374946e-01]
 [2.96731741e-01 5.46400934e-01 3.21005688e-01 1.21375222e-01]
 [9.42325143e-01 8.27738389e-02 6.07352736e-01 3.26864570e-01]
 [5.26871364e-01 3.75543239e-01 6.65994052e-01 2.49188885e-01]]
[[0.94232514 0.08277384 0.60735274 0.32686457]
 [0.         0.52033602 0.12975447 0.01844781]
 [0.         0.         0.24430475 0.05475931]
 [0.         0.         0.         0.        ]]
3


In [132]:
X = matrix_generate(5, 4, type_ = "stagging")
print(X)
X = ForwardGauss(X)
print(X)
print(rank(X))

[[0.25274428 0.77448111 0.91585844 0.20683817 0.18920219]
 [0.         0.         0.79607166 0.42905066 0.81082628]
 [0.         0.         0.         0.19848755 0.10905133]
 [0.         0.         0.         0.         0.54821538]
 [0.         0.         0.         0.         0.        ]]
[[0.25274428 0.77448111 0.91585844 0.20683817 0.18920219]
 [0.         0.         0.79607166 0.42905066 0.81082628]
 [0.         0.         0.         0.19848755 0.10905133]
 [0.         0.         0.         0.         0.54821538]
 [0.         0.         0.         0.         0.        ]]
4


In [133]:
X = matrix_generate(5, 5, type_ = "degenerate")
print(X)
X = ForwardGauss(X)
print(X)
print(rank(X))

[[0.3708758  0.20060358 0.14691943 0.69858419 0.19907192]
 [0.75416902 0.70995022 1.39828982 0.75419417 0.62992002]
 [0.2412802  0.4496591  0.9782271  0.50967425 0.09975294]
 [0.76944895 0.76388229 0.30421677 0.17547276 0.88908317]
 [0.6337397  0.78235581 0.19947174 0.34627783 0.28941142]]
[[ 7.69448950e-01  7.63882292e-01  3.04216769e-01  1.75472758e-01
   8.89083169e-01]
 [-2.77555756e-17  2.10124469e-01  8.82832228e-01  4.54650322e-01
  -1.79041596e-01]
 [ 2.02364856e-17  0.00000000e+00 -6.94759360e-01 -1.29730289e-01
  -3.12323827e-01]
 [-1.07840738e-33  0.00000000e+00  0.00000000e+00  1.80876528e-16
  -1.93269218e-16]
 [-3.08148791e-33  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00]]
5


In [134]:
X = matrix_generate(5, 5, type_ = "upper_triangular")
ForwardGauss(X, rank=True)

(array([[0.38684017, 0.73902513, 0.87219662, 0.37972753, 0.6852934 ],
        [0.        , 0.41755917, 0.95843575, 0.26780604, 0.63032394],
        [0.        , 0.        , 0.5584318 , 0.94101082, 0.07635358],
        [0.        , 0.        , 0.        , 0.55048797, 0.26281097],
        [0.        , 0.        , 0.        , 0.        , 0.52388621]]), 5)

### Задание 4
Реализовать метод Гаусса решения СЛАУ. Использовать данный метод для решения систем различных размеров. Оценить скорость работы метода Гаусса (необходимое количество операций) в зависимости от размера системы аналитически и практически.

In [135]:
def Gauss(A, b):
    
    n = A.shape[0]
    x = np.zeros(n)
    b = b.reshape(n, 1)
    
    if rank(A) < n:
        print('Система имеет бесконечное количество решений')
        return
    
    A = np.concatenate((A, b), axis=1)
    A = ForwardGauss(A)
    for i in range(n - 1, -1, -1):
        tmp = 0.0
        for j in range(i + 1, n):
            tmp += x[j] * A[i, j]
        x[i] = (A[i, n] - tmp) / A[i, i]
    return x

In [136]:
A = matrix_generate(4, 4, type_ = "upper_triangular")
print(A)
b = np.random.random(4).reshape(4, 1)
print(rank(A))
A = np.concatenate((A, b), axis=1)
print(A)
ForwardGauss(A)

[[0.72814863 0.38124357 0.93083745 0.26499178]
 [0.         0.81182254 0.30657469 0.75689942]
 [0.         0.         0.19470402 0.19305829]
 [0.         0.         0.         0.23547792]]
4
[[0.72814863 0.38124357 0.93083745 0.26499178 0.30104928]
 [0.         0.81182254 0.30657469 0.75689942 0.20769934]
 [0.         0.         0.19470402 0.19305829 0.04600021]
 [0.         0.         0.         0.23547792 0.84738739]]


array([[0.72814863, 0.38124357, 0.93083745, 0.26499178, 0.30104928],
       [0.        , 0.81182254, 0.30657469, 0.75689942, 0.20769934],
       [0.        , 0.        , 0.19470402, 0.19305829, 0.04600021],
       [0.        , 0.        , 0.        , 0.23547792, 0.84738739]])

In [137]:
B = np.array([
    [1, 3, 1],
    [0, 2, 0],
    [0, 0, 1]
], dtype=float)
f = np.array([2, 1, 1], dtype=float)
Gauss(B, f)

array([-0.5,  0.5,  1. ])

In [138]:
A = matrix_generate(4, 4)
print(rank(A))
b = np.random.random(4).reshape(4, 1)
Gauss(A, b)

2
Система имеет бесконечное количество решений


In [139]:
A = matrix_generate(7, 7, type_ = "upper_triangular")
print(rank(A))
b = np.random.random(7).reshape(7, 1)
Gauss(A, b)

7


array([-0.57681256, -2.39209118,  0.52421957, -3.12710352,  4.43064798,
       -0.82750846,  0.96941895])